In [1]:
import pandas as pd

df = pd.read_csv('cleaned_merged_doctors_symptomsss.csv')

df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Display the first few rows
df.head()

,Disease,Disease_Category,Symptoms,CommonAgeGroup,Sex,Severity,Specialist,SyntheticTreatment,Name,Category,Address/Details,City,Rating,Mapped_Category
0,appendicitis,surgeon,loss appetite abdominal pain,Child,Any,Mild,Surgeon,Consultation and standard treatment protocols....,Dr Imran Khan Orthopaedic in Anantnag J&K,orthopedic surgeon,"Orthopedic surgeon Valley pharmacy, opp. GMC",Neelum,4.8,surgeon
1,appendicitis,surgeon,loss appetite abdominal pain,Teen,Any,Moderate,Surgeon,Consultation and standard treatment protocols....,Dr Lakshay Goel,orthopedic surgeon,"Sports medicine clinic Hospital, Metro Station...",Kohat,4.9,surgeon
2,cerebrovascular accident,neurologist,vision problem trouble walking numbness delirium,Teen,Any,Moderate,Neurologist,Consultation and standard treatment protocols....,Dr ijaz Alam neurologist clinic,neurologist,Neurologist Q965+25P Swat international hospit...,Swat,5.0,neurologist
3,diabetes,endocrinologist,lassitude increased thirst blurred vision freq...,Teen,Male,Moderate,Endocrinologist,"Lifestyle changes, regular medication, and per...",Dr Shahzad Hussain,"the heart and diabetes clinic, endocrinologist","Diabetologist 18 Nishtar Road Zainab palaza, n...",Shujabad,5.0,endocrinologist
4,asthma,pulmonologist,shortness breath cough,Elderly,Any,Moderate,Pulmonologist,"Lifestyle changes, regular medication, and per...",Dr Iltifat Sultan's Clinic,pulmonologist,Pulmonologist 687-B Satayana Rd,Samundri,3.6,pulmonologist


In [13]:
!pip install sentence-transformers
!pip install hnswlib


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp311-cp311-linux_x86_64.whl size=2389213 sha256=7735080bec08702f7b2c63956fb638c1714a2048e81b152ab0556f542495ab5c
  Stored in directory: /root/.cache/pip/wheels/ea/4e/27/39aebca9958719776e36fada290845a7ef10f053ad70e22ceb
Successfully built hnswlib


In [14]:
# Install dependencies (only run once)
!pip install -q sentence-transformers hnswlib

# 1. Import libraries
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import hnswlib

# 2. Load your data (assuming df already loaded)

# 3. Merge key text fields
df['merged_text'] = df['Symptoms'] + ' ' + df['Disease'] + ' ' + df['SyntheticTreatment'] + ' ' + df['Specialist']

# 4. Load the pretrained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# 5. Generate embeddings
embeddings = model.encode(df['merged_text'].tolist(), show_progress_bar=True)

# 6. Initialize HNSW index
dim = embeddings.shape[1]
num_elements = len(embeddings)

hnsw_index = hnswlib.Index(space='cosine', dim=dim)
hnsw_index.init_index(max_elements=num_elements, ef_construction=200, M=16)
hnsw_index.add_items(embeddings, np.arange(num_elements))
hnsw_index.set_ef(50)

# 7. Query the index
query_embedding = embeddings[0]
labels, distances = hnsw_index.knn_query(query_embedding, k=5)

# 8. Show results
print("Query:", df['merged_text'][0])
print("\nTop 5 similar records:")
for i, idx in enumerate(labels[0]):
    print(f"{i+1}. {df['merged_text'][idx]} (Distance: {distances[0][i]:.4f})")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/738 [00:00<?, ?it/s]

Query: loss appetite queasiness appendicitis Consultation and standard treatment protocols. Stronger prescription drugs may be necessary Surgeon

Top 5 similar records:
1. loss appetite queasiness appendicitis Consultation and standard treatment protocols. Stronger prescription drugs may be necessary Surgeon (Distance: 0.0000)
2. loss appetite queasiness appendicitis Consultation and standard treatment protocols. Stronger prescription drugs may be necessary Surgeon (Distance: 0.0000)
3. loss appetite queasiness appendicitis Consultation and standard treatment protocols. Stronger prescription drugs may be necessary Surgeon (Distance: 0.0000)
4. loss appetite queasiness appendicitis Consultation and standard treatment protocols. Stronger prescription drugs may be necessary Surgeon (Distance: 0.0000)
5. loss appetite queasiness appendicitis Consultation and standard treatment protocols. Stronger prescription drugs may be necessary Surgeon (Distance: 0.0000)


In [16]:
# Save embeddings with merged_text in a single CSV

import pandas as pd
import numpy as np

# Assuming 'embeddings' and 'df' (with 'merged_text') are already available

# Step 1: Convert embeddings to DataFrame
embeddings_df = pd.DataFrame(embeddings)

# Step 2: Add the original merged_text column
embeddings_df['merged_text'] = df['merged_text'].values

# Step 3: Save to CSV
embeddings_df.to_csv('embeddings_with_text.csv', index=False)

print("✅ Embeddings with merged_text saved successfully to 'embeddings_with_text.csv'")


✅ Embeddings with merged_text saved successfully to 'embeddings_with_text.csv'


In [17]:
embeddings_df

,0,1,2,3,4,5,6,7,8,9,...,375,376,377,378,379,380,381,382,383,merged_text
0,0.067780,-0.019581,0.027891,0.005417,-0.132982,-0.064865,0.009591,0.104205,0.014645,-0.041661,...,0.007168,0.020552,-0.002789,0.035411,-0.003905,0.002368,-0.030133,-0.034819,0.032954,loss appetite queasiness appendicitis Consulta...
1,0.002449,0.084551,0.067195,-0.007846,0.057168,0.034707,0.047531,0.065230,-0.107430,-0.023815,...,-0.028436,-0.001993,0.010754,0.019279,0.032318,-0.041302,-0.094305,-0.018032,0.081371,red patch itchy skin edema eczema Consultation...
2,0.008797,0.098840,0.097749,0.034257,0.045046,-0.008098,0.008618,0.040685,-0.119651,-0.044987,...,-0.003215,0.001757,0.020641,0.021483,-0.012747,-0.034317,-0.045247,0.003779,0.051655,dryness edema eczema Consultation and standard...
3,0.074995,0.024489,-0.012961,-0.005276,-0.000430,-0.041336,-0.069398,0.015965,-0.047688,-0.078393,...,0.063118,-0.071086,-0.026408,0.050151,0.002017,-0.032620,-0.106187,-0.001149,0.088913,cough chest tightness shortness breath wheezin...
4,0.024335,0.009320,0.003181,0.063697,-0.048778,-0.046102,0.051239,0.024591,-0.037199,-0.031355,...,0.024436,-0.040005,-0.026552,0.021365,-0.017529,0.060583,-0.102572,-0.063801,-0.053244,frequent urination lassitude diabetes Lifestyl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23603,-0.046749,0.050771,-0.029103,0.074508,-0.051599,0.021357,0.013635,0.057503,-0.052808,-0.016969,...,-0.016539,0.034408,-0.073051,-0.015236,-0.004329,-0.078900,-0.108301,0.044332,0.052922,"acne weight gain cos Hormonal therapy, dietary..."
23604,0.075132,-0.005410,0.019485,-0.009364,-0.124746,-0.095822,-0.028605,0.082852,0.022597,-0.044229,...,-0.026872,-0.001606,-0.020071,0.050534,0.019404,-0.002928,-0.058327,-0.024048,0.008054,loss appetite abdominal pain appendicitis Cons...
23605,0.038670,0.109182,0.065942,0.026304,0.076845,0.011726,0.062574,0.056635,-0.159934,-0.042123,...,0.001987,0.004568,0.032321,0.011879,-0.031435,-0.034787,-0.090094,0.001895,0.055809,itchy skin dryness eczema Consultation and sta...
23606,0.066794,0.068660,0.075667,0.102919,0.083553,-0.028517,-0.079386,0.011553,-0.039923,-0.060680,...,0.052580,-0.014083,0.094410,0.013074,-0.005463,0.030873,-0.070154,0.001259,0.065820,chest pain shortness breath sweating queasines...
